In [1]:
import torch
import numpy as np

In [26]:
def getPick(p):
    if p.ndim != 1:
        raise Exception("ndim does not equal 1！");
    lp = p.shape[0]
    
    def pick(i):
        if lp == 0:
            return np.array([], dtype='int16')
        mx = p.max();
        mn = p.min();
        if i.ndim != 1:
            raise Exception("ndim does not equal 1！");  
        
        if mx >= i.shape[0]:
            raise Exception("Argument invalid！");
        j = np.zeros(lp, dtype = 'int16');
        for k in range(lp):
            j[k]= i[p[k]];
        return j
    return pick

def traversal(traversalShape,action):      
    ilen = len(traversalShape)
    if ilen == 0:
        raise Exception("traversalShape's length == 0！");
    elif traversalShape[0] == 0:
        raise Exception("traversalShape's length == 0！");
    traversalInd = np.array(traversalShape);
    def traversalForInner(i):   
        n = traversalShape[i];
        
        if i < ilen-1:
            for j in range(n):
                traversalInd[i] = j;
                traversalForInner(i+1);
        elif i == ilen-1:
            for j in range(n):
                traversalInd[i] = j;
                action(traversalInd);
        else:
            raise Exception("Exception raised！");
    traversalForInner(0)

def getProvisionXTransformer(shape0, shape1, T0, p0, p1, p):    
    if p0.dtype != 'int16' or p0.dtype != 'int16' or p.dtype != 'int16':
        raise Exception("Data type error！");
    elif p0.ndim != 1 or p0.ndim != 1 or p.ndim != 1:
        raise Exception("ndim to form pick does not equal 1！");
    E = np.zeros(shape0 + (len(shape1),), dtype = 'int16');
    I = np.zeros(len(shape0), dtype = 'int16');
    J = np.zeros(len(shape1), dtype = 'int16');
    pick0 = getPick(p0);
    pick1 = getPick(p1);
    pick = getPick(p);
    single = False;
    if T0.ndim == len(p0):
        single = True;
    def combine(IC):
        print(T0)
        J0 = pick0(IC);
        if single:
            Jp = [T0[tuple(J0)]]
        else:
            Jp = T0[tuple(J0)];
        #Jp = T0[tuple(J0)] 
        J1 = pick1(IC)
        Jl = list(tuple(Jp) + tuple(J1))
        Ja = np.array(Jl, dtype = 'int16')
        J = pick(Ja)
        E[tuple(IC)] = J;
    traversal(shape0,combine)
   
    return E

def scatter(src, X, XTransformer):
    shape = XTransformer.shape[:-1];    
    I = np.zeros(len(shape), dtype = 'int16');
    def action(Ia):
        src[tuple(XTransformer[tuple(Ia)])] = X[tuple(Ia)]
    traversal(shape,action)
    return src

def scatterX(src, X, T0, p0, p1, p):
    shape0 = X.shape;
    shape1 = src.shape;
    XTransformer = getProvisionXTransformer(shape0, shape1, T0, p0, p1, p);
    
    scatter(src, X, XTransformer)
    return src
def tensorflowScatter(tensor, indices, updates):
    l = list(range(len(indices.shape) - 1))
   
    tl = indices.shape[-1]
    ln = list(range(len(tensor.shape)))
    
    t1 = ln[tl:] 
    p0 = np.array(l, dtype='int16')
    p1 = np.array(t1, dtype='int16')
    p = np.array(ln, dtype='int16')
    scatterX(tensor, updates, indices, p0, p1, p)
def pytorchScatter

In [6]:
#test case 0 
u = torch.tensor([[0.4707, 0.5464, 0.4439, 0.7922, 0.8641],
        [0.5085, 0.5926, 0.2702, 0.3165, 0.6066]]);
torch.zeros(3, 5).scatter_(1, torch.tensor([[0],[2]]), u)

tensor([[0.4707, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5085, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [38]:
#test case 0 pytorch
u = np.array([[0.4707, 0.5464, 0.4439, 0.7922, 0.8641],
        [0.5085, 0.5926, 0.2702, 0.3165, 0.6066]]);
ind = np.array([[0],[2]], dtype='int16');
src = np.zeros((3, 5))
l = len(ind.shape)
ls = list(range(l))
dim =1
p1 = np.array(tuple(ls[:dim])+tuple(ls[(dim+1):]), dtype='int16');
p0 = np.array(list(range(l)), dtype='int16');
p = np.array(tuple(ls[1:(dim+1)])+(0,)+tuple(ls[(dim+1):]), dtype='int16');
u2 = np.zeros(ind.shape);
def f(I):
    print(I)
    u2[tuple(I)] = u[tuple(I)]
traversal(ind.shape,f)
#print("ind2")
print(u2)
scatterX(src, u2, ind, p0, p1, p)

[0 0]
[1 0]
[[0.4707]
 [0.5085]]
[[0]
 [2]]
[[0]
 [2]]


array([[0.4707, 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.5085, 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    ]])

In [37]:
l=[2,3]
l[2:]

[]

In [11]:
tuple(1)

TypeError: 'int' object is not iterable

In [12]:
len(1)

TypeError: object of type 'int' has no len()

In [13]:
1.tolist

SyntaxError: invalid syntax (<ipython-input-13-05fc7306edea>, line 1)

In [14]:
list(1)

TypeError: 'int' object is not iterable